In [ ]:
import itertools
import sys
import traceback
from pathlib import Path
import yaml

from astrocast.denoising import Network, PyTorchNetwork, SubFrameDataset, SubFrameGenerator

# General considerations

This training is generally a one-time requirement; once completed, the model can be reapplied to subsequent datasets without the need for retraining. The training phase can range from 1 to 12 hours, depending on the complexity of the data and computational resources. The architecture used in astroCAST is modeled on the suggested network published in [Lecoq et. al. 2021](https://doi.org/10.1038/s41592-021-01285-2). 

### Table 1: Parameters for denoiser training

| Parameter               | Default value         | Comment                                                                                                         |
|-------------------------|-----------------------|-----------------------------------------------------------------------------------------------------------------|
| **Input Data**          |                       |                                                                                                                 |
| `training_files`        | `path/to/training/files` | Path to your training files. Accepts individual file paths or multiple files (e.g., `./train_*.h5`).           |
| `validation_files`      |                       | Path to your validation files, similar to training files.                                                       |
| `loc`                   | `dataset_name`        | Specifies the location inside `.h5` files.                                                                      |
| `batch_size` / `batch_size_val` | `32`          | Number of data samples processed at once. Can be reduced if memory is a constraint.                             |
| `max_per_file` / `max_per_val_file` | `16`      | Limits the number of training samples per video to avoid overtraining. Aim for one to few minutes per training epoch. |
| **Data Transformation** |                       |                                                                                                                 |
| `train_rotation`        | `(1, 2, 3)`           | Allows rotating training images to increase data variety. Rotation in steps of 90°.                             |
| `train_flip`            | `(0, 1)`              | Allows flipping training images, again for variety.                                                             |
| `normalize`             | `global`              | Controls scaling of input data. Global normalization is recommended.                                            |
| `batch_normalize`       | `False`               | Optional feature to further scale data during training.                                                         |
| **Denoiser Architecture** |                     |                                                                                                                 |
| `input_size`            | `(256, 256)`          | Size of the area denoised at once. Should ideally match your frame size.                                        |
| `n_stacks`              | `2`                   | Layers in the encoder and decoder. More layers might improve quality but also increase complexity.              |
| `kernel`                | `32`                  | Complexity of the initial layer. Higher values may improve results but increase risk of overfitting.            |
| `pre_post_frames`       | `5`                   | Frames around the target frame used for denoising. More frames might improve quality but increase requirements.  |
| `gap_frames`            | `0`                   | Skips frames immediately before and after the target frame for better denoising in some cases.                  |
| **Training Parameters** |                       |                                                                                                                 |
| `epochs`                | `10`                  | Maximum training cycles. Can be high if using early stopping.                                                   |
| `loss`                  | `annealed_loss`       | Loss function used to assess reconstruction quality.                                                           |
| `learning_rate`         | `0.001`               | Speed of model learning. Too high values can make the model unstable.                                           |
| `decay_rate`, `decay_steps` | `0.99`, `250`     | Gradually reduces the learning rate for stability.                                                              |
| `patience`              | `3`                   | Stops training if no improvement after this many cycles.                                                        |
| `min_delta`             | `0.001`               | What's considered an "improvement" in model performance.                                                        |
| `pretrained_weights`    | `None`                | Use weights from a previous model to speed up training or for transfer learning.                                |
| `use_cpu`               | `True`                | Use CPU or GPU for training. GPU is faster for training.                                                        |
| `in_memory`             | `False`               | Toggle if training data is loaded into memory. Not recommended due to high memory usage.                        |
| **Output Parameters**   |                       |                                                                                                                 |
| `save_path`             | `None`                | Where to save your trained model.                                                                               |

### **CRITICAL** considerations

#### Architecture
The settings of the *Denoiser Architecture* cannot be changed without training a new model from scratch.

#### Overtraining
Overtraining is a significant concern when using the denoiser, as it employs a neural network internally. To mitigate this risk, the denoiser incorporates an automatic stopping mechanism that halts training if no further improvement is observed on the validation dataset. Nevertheless, caution must be exercised when selecting a high value for n_stacks or using a limited training dataset, as these factors can still increase the likelihood of overtraining. If overtraining does occur, it may result in high denoising quality for the training data but poor generalization for new data. 


# Load configurations

In [ ]:
yaml_file = Path(".").absolute().parent.parent.joinpath("scripts/generate_pretrain_models.yaml")

with open(yaml_file, "r") as file:
    config = yaml.safe_load(file)

display(config)

In [ ]:
params = config["param"]
use_pytorch = config["use_pytorch"]
use_cpu = config["use_cpu"]

root = Path(config["root_path"])
model_path = Path(config["model_path"])

# change configurations if desired
# use_pytorch = False

## Set up folders for training and saving models

In [ ]:
# change to your data directory
my_data_directory = Path("/media/janrei1/data/deep")

assert my_data_directory.exists(), f"can't find data directory: {my_data_directory}"

root = my_data_directory.joinpath(root)
if not root.is_dir():
    root.mkdir(parents=True)
print(f"loading data from: {root}")

# Optional: download training files or provide your own
# !astrocast download_datasets {{my_data_directory}}

model_path = my_data_directory.joinpath(model_path)
if not model_path.is_dir():
    model_path.mkdir(parents=True)
print(f"saving models to: {model_path}")

# Train models
## Create all possible configuration combinations


In [ ]:
# Convert single values to lists and keep lists/tuples as-is
params = {k: [v] if not isinstance(v, (list, tuple)) else v for k, v in params.items()}

# Generate all combinations
keys, values = zip(*params.items())
combinations = [dict(zip(keys, prod)) for prod in itertools.product(*values)]


## iterate over combinations and train

In [ ]:
# Loop over all combinations
for param_set in combinations:
    
    name_parts = []
    for key, value in param_set.items():
        
        if key in ["epochs", "patience", "min_delta"]:
            continue
        
        if isinstance(value, (list, tuple)):
            value_str = '_'.join(map(str, value))
        else:
            value_str = str(value)
        
        name_parts.append(f"{key}_{value_str}")
    name = '-'.join(name_parts)
    
    for k, v in config["data"].items():
        
        print(f"{k}:{name}")
        
        save_model_path = model_path.joinpath(f"{k}_{name}")
        if save_model_path.joinpath("model.h5").is_file():
            print(f"Skipping > model exists: {save_model_path}")
            continue
        
        try:
            input_size = param_set["input_size"]
            pre_post_frames = param_set["pre_post_frames"]
            gap_frames = param_set["gap_frames"]
            train_rotation = param_set["train_rotation"]
            
            n_stacks, kernel = param_set["architecture"]
            epochs = param_set["epochs"]
            patience = param_set["patience"]
            min_delta = param_set["min_delta"]
            
            # Trainer
            train_str = v["train"]
            if "*" in train_str:
                train_paths = list(root.glob(train_str))
            else:
                train_paths = root.joinpath(train_str)
            
            # Validator
            val_paths = None
            if "val" in v:
                val_str = v["train"]
                if "*" in val_str:
                    val_paths = list(root.glob(val_str))
                else:
                    val_paths = root.joinpath(val_str)
            
            if not use_pytorch:
                
                train_gen = SubFrameGenerator(paths=train_paths, max_per_file=v["max_per_file"], loc=v["loc"],
                                              input_size=input_size,
                                              pre_post_frames=pre_post_frames, gap_frames=gap_frames,
                                              allowed_rotation=train_rotation,
                                              padding=None, batch_size=8, normalize="global", in_memory=False,
                                              allowed_flip=[0, 1], shuffle=True)
                
                # Validator
                if val_paths is not None:
                    
                    val_gen = SubFrameGenerator(
                            paths=val_paths, max_per_file=3, loc=v["loc"], input_size=input_size,
                            pre_post_frames=pre_post_frames, gap_frames=gap_frames, allowed_rotation=[0],
                            padding=None, batch_size=16, normalize="global", in_memory=False,
                            cache_results=True,
                            allowed_flip=[-1], shuffle=True)
                
                else:
                    val_gen = None
                
                # Network
                
                net = Network(train_generator=train_gen, val_generator=val_gen, learning_rate=0.001, decay_rate=0.99,
                              pretrained_weights=None,
                              n_stacks=n_stacks, kernel=kernel,
                              batchNormalize=False, use_cpu=use_cpu)
                
                net.run(batch_size=1, num_epochs=epochs, patience=patience, min_delta=min_delta,
                        save_model=save_model_path)
            
            else:
                
                train_dataset = SubFrameDataset(paths=train_paths, input_size=input_size, loc=v["loc"],
                                                pre_post_frames=pre_post_frames, max_per_file=v["max_per_file"],
                                                gap_frames=gap_frames, allowed_rotation=train_rotation, padding=None,
                                                normalize="global", in_memory=False, allowed_flip=[0, 1], shuffle=True)
                
                val_dataset = None
                if "val" in v:
                    val_dataset = SubFrameDataset(paths=val_paths, input_size=input_size, loc=v["loc"],
                                                  pre_post_frames=pre_post_frames, max_per_file=3,
                                                  gap_frames=gap_frames, allowed_rotation=0, padding=None,
                                                  normalize="global", in_memory=False, allowed_flip=-1, shuffle=True)
                
                net = PyTorchNetwork(train_dataset, val_dataset=val_dataset, batch_size=16, shuffle=True, num_workers=4,
                                     learning_rate=0.001, momentum=0.9, decay_rate=0.1, decay_steps=30,
                                     n_stacks=n_stacks, kernels=kernel, kernel_size=3, batch_normalize=False,
                                     use_cpu=use_cpu)
                
                net.run(num_epochs=epochs, save_model=save_model_path, patience=patience, min_delta=min_delta)
        
        except KeyboardInterrupt:
            sys.exit(2)
        
        except Exception as err:
            print(f"Error in {k}:{name}: {err}")
            traceback.print_exc()